In [2]:
import argparse

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [48]:
test = 12
N = 860

In [49]:
df_whole = pd.read_csv(str("/home/gean/Code/nns_performance_prediction/results/fast/test" + str(test) + 
                           "/N" + str(N) + "_nasbench_metrics.csv"))
# df_whole.reset_index()
df_whole.head()

,LinearRegression_MAE,SGDRegressor_MAE,Lasso_MAE,BayesianRidge_MAE,KNeighborsRegressor_MAE,DecisionTreeRegressor_MAE,SVR_MAE,MLPRegressor_MAE,RandomForestRegressor_MAE,AdaBoostRegressor_MAE,...,Lasso_R2,BayesianRidge_R2,KNeighborsRegressor_R2,DecisionTreeRegressor_R2,SVR_R2,MLPRegressor_R2,RandomForestRegressor_R2,AdaBoostRegressor_R2,GradientBoostingRegressor_R2,DummyRegressor_R2
0,0.116441,0.118683,0.122062,0.116591,0.116288,0.118069,0.115629,0.116435,0.115148,0.116307,...,-0.000998,0.059229,0.045133,0.041740,0.059848,0.057407,0.074392,0.056553,0.062893,-0.000998
1,0.160273,0.161708,0.170143,0.160425,0.156332,0.158098,0.157975,0.158720,0.153939,0.159296,...,-0.000422,0.104897,0.128674,0.109804,0.119002,0.122718,0.154168,0.124025,0.161663,-0.000422
2,0.051569,0.052420,0.056112,0.051581,0.050484,0.051216,0.070739,0.048916,0.049776,0.051269,...,-0.000017,0.123368,0.158551,0.132147,-0.027270,-0.000509,0.170385,0.146518,0.187300,-0.000017
3,0.020279,0.027198,0.024451,0.020253,0.020310,0.017868,0.058069,0.021002,0.016777,0.019892,...,-0.000189,0.112948,0.129301,0.113266,-0.630525,0.100858,0.150593,0.164615,0.160521,-0.000189
4,0.118428,0.118558,0.123371,0.118885,0.118436,0.118367,0.116423,0.119112,0.117280,0.119475,...,-0.000767,0.053728,0.049122,0.056253,0.062953,0.059699,0.064334,0.043190,0.058691,-0.000767


In [50]:
formatted_cols = {'Seed': [], 'Subset': [], 'Model': [], 'MAE': [], 'MSE': [], 'R2': []}

reg_cols = ['LinearRegression', 'SGDRegressor', 'Lasso', 'BayesianRidge', 'KNeighborsRegressor', 
            'DecisionTreeRegressor', 'SVR', 'MLPRegressor', 'RandomForestRegressor', 'AdaBoostRegressor',
            'GradientBoostingRegressor', 'DummyRegressor']

i = 0
for seed in [0, 1, 10, 42, 100, 123, 666, 1000, 1234, 12345]:
    for count, subset in enumerate([4, 12, 36, 108]):
        for reg in reg_cols:
            
            formatted_cols['Seed'].append(seed)
            formatted_cols['Subset'].append(subset)
            formatted_cols['Model'].append(reg)
            formatted_cols['MAE'].append(df_whole[str(reg + '_MAE')][i + count])
            formatted_cols['MSE'].append(df_whole[str(reg + '_MSE')][i + count])
            formatted_cols['R2'].append(df_whole[str(reg + '_R2')][i + count])
            
    i = i + 4

In [51]:
df_results = pd.DataFrame.from_dict(formatted_cols)
df_results.to_csv(str("/home/gean/Code/nns_performance_prediction/results/fast/test" + str(test) + 
                           "/N" + str(N) + "_nasbench_metrics_formatted.csv"), index=False, float_format='%.6f')

# df_results = df_results.astype({'Seed': np.int16, 'Subset': np.int16, 'MAE': np.float32, 'MSE': np.float32, 'R2': np.float32})
df_results.head()

,Seed,Subset,Model,MAE,MSE,R2
0,0,4,LinearRegression,0.116441,0.019230,0.057961
1,0,4,SGDRegressor,0.118683,0.019333,0.052922
2,0,4,Lasso,0.122062,0.020434,-0.000998
3,0,4,BayesianRidge,0.116591,0.019204,0.059229
4,0,4,KNeighborsRegressor,0.116288,0.019492,0.045133


In [52]:
df_means_stds = {'Subset': [], 'Model': [], 'MAE_mean': [], 'MAE_std': [], 'MSE_mean': [], 'MSE_std': [], 'R2_mean': [], 'R2_std': []}

In [53]:
reg_cols = ['LinearRegression', 'SGDRegressor', 'Lasso', 'BayesianRidge', 'KNeighborsRegressor', 
            'DecisionTreeRegressor', 'SVR', 'MLPRegressor', 'RandomForestRegressor', 'AdaBoostRegressor',
            'GradientBoostingRegressor', 'DummyRegressor']

for subset in [4, 12, 36, 108]:
    for reg in reg_cols:

        df_aux = df_results.loc[(df_results['Model'] == reg) & (df_results['Subset'] == subset)]
        df_means_stds['Subset'].append(subset)
        df_means_stds['Model'].append(reg)
        df_means_stds['MAE_mean'].append(df_aux['MAE'].mean())
        df_means_stds['MAE_std'].append(df_aux['MAE'].std())
        df_means_stds['MSE_mean'].append(df_aux['MSE'].mean())
        df_means_stds['MSE_std'].append(df_aux['MSE'].std())
        df_means_stds['R2_mean'].append(df_aux['R2'].mean())
        df_means_stds['R2_std'].append(df_aux['R2'].std())
            
df_final_results = pd.DataFrame.from_dict(df_means_stds)
df_final_results.to_csv("/home/gean/Code/nns_performance_prediction/results/fast/test" + str(test) + 
                           "/N" + str(N) + "_nasbench_metrics_mean_std.csv", index=False, float_format='%.6f')
df_final_results

,Subset,Model,MAE_mean,MAE_std,MSE_mean,MSE_std,R2_mean,R2_std
0,4,LinearRegression,0.117787,0.000668,0.019302,0.000099,0.054404,0.004857
1,4,SGDRegressor,0.117671,0.001084,0.019357,0.000126,0.051694,0.006209
2,4,Lasso,0.122847,0.000506,0.020422,0.000011,-0.000477,0.000542
3,4,BayesianRidge,0.118097,0.000727,0.019321,0.000090,0.053431,0.004399
4,4,KNeighborsRegressor,0.117776,0.000752,0.019792,0.000324,0.030340,0.015880
5,4,DecisionTreeRegressor,0.118363,0.000645,0.019669,0.000353,0.036414,0.017286
6,4,SVR,0.116322,0.000892,0.019251,0.000185,0.056860,0.009076
7,4,MLPRegressor,0.121222,0.012662,0.020976,0.004126,-0.027617,0.202180
8,4,RandomForestRegressor,0.116815,0.000985,0.019193,0.000231,0.059699,0.011347
9,4,AdaBoostRegressor,0.118637,0.001088,0.019397,0.000148,0.049712,0.007235


# GCN

In [22]:
df_means_stds = {'Subset': [], 'N': [], 'Model': [], 'MAE_mean': [], 'MAE_std': [], 'MSE_mean': [], 'MSE_std': [], 
                 'R2_mean': [], 'R2_std': [], 'Time_mean': [], 'Time_std': []}

for subset in [4, 12, 36, 108]:
    df_gcn = pd.read_csv(str("/home/gean/Downloads/compilado_test/runs_p_test/runs_" + str(subset) + ".csv"))
    
    for train_size in [43, 86, 129, 172, 344, 860]:
        df_aux = df_gcn.loc[df_gcn['train_size'] == train_size]
        
        if len(df_aux) > 0:
            df_means_stds['Subset'].append(subset)
            df_means_stds['N'].append(train_size)
            df_means_stds['Model'].append("GCN_Complete")
            df_means_stds['MAE_mean'].append(df_aux['mae'].mean())
            df_means_stds['MAE_std'].append(df_aux['mae'].std())
            df_means_stds['MSE_mean'].append(df_aux['mse'].mean())
            df_means_stds['MSE_std'].append(df_aux['mse'].std())
            df_means_stds['R2_mean'].append(df_aux['r2'].mean())
            df_means_stds['R2_std'].append(df_aux['r2'].std())
            df_means_stds['Time_mean'].append(df_aux['time'].mean())
            df_means_stds['Time_std'].append(df_aux['time'].std())
            
df_final_results = pd.DataFrame.from_dict(df_means_stds)
df_final_results.to_csv("/home/gean/Code/nns_performance_prediction/results/fast/GCN_Complete/GCN_Complete_nasbench_metrics_mean_std_time.csv", 
                        index=False, float_format='%.6f')
df_final_results

,Subset,N,Model,MAE_mean,MAE_std,MSE_mean,MSE_std,R2_mean,R2_std,Time_mean,Time_std
0,4,43,GCN_Complete,0.127250,0.001125,0.022908,0.000325,-0.003170,0.013619,3.580985,0.038976
1,4,86,GCN_Complete,0.140314,0.013782,0.025666,0.004912,-0.123656,0.213307,4.475876,0.363717
2,4,129,GCN_Complete,0.182662,0.012841,0.043940,0.006249,-0.925549,0.274770,7.148843,0.953450
3,4,172,GCN_Complete,0.170792,0.019419,0.038118,0.009516,-0.670698,0.418065,15.536832,1.761750
4,4,344,GCN_Complete,0.159759,0.014790,0.033113,0.004928,-0.451107,0.216733,155.577310,11.318715
5,4,860,GCN_Complete,0.166691,0.002650,0.035530,0.001167,-0.557876,0.050626,501.506173,21.713110
6,12,43,GCN_Complete,0.178489,0.002307,0.042682,0.000698,-0.009155,0.016142,3.507684,0.098497
7,12,86,GCN_Complete,0.177941,0.001200,0.044485,0.002664,-0.051965,0.062790,4.482705,0.139870
8,12,129,GCN_Complete,0.186843,0.012609,0.053433,0.011672,-0.265230,0.276834,6.955229,1.643942
9,12,172,GCN_Complete,0.190905,0.011933,0.057275,0.010147,-0.355172,0.240338,14.229026,2.843279
